In [4]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns

In [13]:
df = pd.read_excel("Data/circuit_outages_ibec.xlsx", sheet_name = "Circuit Outage", usecols= "A:G")
df.head()

,Circuit Number,Circuit Name,Outage Date,Outage CauseCode,Outage Cause,Outage Duration (min),Customers Affected
0,1,Orange,2024-01-05,8,Other,837,376
1,1,Orange,2024-08-27,3,Weather,515,1299
2,1,Orange,2024-11-10,4,Third Party,1169,862
3,2,Alabama,2024-03-13,1,UG Equipment Failure,227,1399
4,2,Alabama,2024-03-15,1,UG Equipment Failure,19,3198


In [14]:
df.columns

Index(['Circuit Number', 'Circuit Name', 'Outage Date', 'Outage CauseCode',
       'Outage Cause', 'Outage Duration (min)', 'Customers Affected'],
      dtype='object')

In [15]:
df.rename(columns = {'Circuit Number':'circuit_number', 'Circuit Name':'circuit_name', 'Outage Date':'outage_date', 'Outage CauseCode':'outage_cause_code',
                    'Outage Cause':'outage_cause', 'Outage Duration (min)':'outage_duration', 'Customers Affected':'customers_affected'})

,circuit_number,circuit_name,outage_date,outage_cause_code,outage_cause,outage_duration,customers_affected
0,1,Orange,2024-01-05,8,Other,837,376
1,1,Orange,2024-08-27,3,Weather,515,1299
2,1,Orange,2024-11-10,4,Third Party,1169,862
3,2,Alabama,2024-03-13,1,UG Equipment Failure,227,1399
4,2,Alabama,2024-03-15,1,UG Equipment Failure,19,3198
...,...,...,...,...,...,...,...
95,21,Dinan,2024-04-14,7,OH Equipment Failure,758,2475
96,21,Dinan,2024-04-24,4,Third Party,727,1043
97,21,Dinan,2024-05-28,1,UG Equipment Failure,31,2562
98,21,Dinan,2024-07-29,4,Third Party,1221,644


In [21]:
df_circuit_lookup = pd.read_excel("Data/circuit_outages_ibec.xlsx", sheet_name = "LookUp", usecols= "A:J")
df_outage_cause_lookup = pd.read_excel("Data/circuit_outages_ibec.xlsx", sheet_name = "LookUp", usecols= "L:M", nrows=8)

In [22]:
df_circuit_lookup

,Circuit Number,Circuit Name,KV,Region,Customer Count,Circuit Miles,% Overhead,% Underground,2023 SAIDI,2023 SAIFI
0,1,Orange,4,Coastal,1299,45,90,10,1.68,0.002
1,2,Alabama,12,North,3447,29,82,18,0.57,0.001
2,3,Green,12,Coastal,2701,26,55,45,2.40,0.002
3,4,Oregon,16,Desert,1147,101,85,15,1.39,0.002
4,5,Yellow,4,North,1706,43,85,15,2.20,0.005
5,6,Washington,12,North,1312,38,18,82,1.79,0.002
6,7,Lincoln,16,Mountain,1634,95,65,35,2.40,0.004
7,8,Johnson,16,Mountain,1641,64,60,40,0.91,0.001
8,9,Magenta,16,Mountain,2367,129,75,25,3.77,0.006
9,10,Adams,4,Desert,1647,32,77,23,1.08,0.001


In [23]:
df_outage_cause_lookup

,Outage Cause Code,Outage Cause
0,1,UG Equipment Failure
1,2,Vegetation
2,3,Weather
3,4,Third Party
4,5,Animal
5,6,Operation
6,7,OH Equipment Failure
7,8,Other


In [29]:
df_circuit_lookup.columns
df_circuit_lookup.rename(columns={'Circuit Number':'circuit_number', 'Circuit Name':'circuit_name', 'KV':'voltage', 'Region':'region', 'Customer Count':'customer_count',
       'Circuit Miles':'circuit_miles', '% Overhead':'percent_overhead', '% Underground':'percent_undergound', '2023 SAIDI':'avg_duration_outage_per_customer',
       '2023 SAIFI':'outage_freq_per_365_days'})

,circuit_number,circuit_name,voltage,region,customer_count,circuit_miles,percent_overhead,percent_undergound,avg_duration_outage_per_customer,outage_freq_per_365_days
0,1,Orange,4,Coastal,1299,45,90,10,1.68,0.002
1,2,Alabama,12,North,3447,29,82,18,0.57,0.001
2,3,Green,12,Coastal,2701,26,55,45,2.40,0.002
3,4,Oregon,16,Desert,1147,101,85,15,1.39,0.002
4,5,Yellow,4,North,1706,43,85,15,2.20,0.005
5,6,Washington,12,North,1312,38,18,82,1.79,0.002
6,7,Lincoln,16,Mountain,1634,95,65,35,2.40,0.004
7,8,Johnson,16,Mountain,1641,64,60,40,0.91,0.001
8,9,Magenta,16,Mountain,2367,129,75,25,3.77,0.006
9,10,Adams,4,Desert,1647,32,77,23,1.08,0.001


In [30]:
df_outage_cause_lookup.columns
df_outage_cause_lookup.rename(columns={'Outage Cause Code':'outage_cause_code', 'Outage Cause':'outage_cause'})

,outage_cause_code,outage_cause
0,1,UG Equipment Failure
1,2,Vegetation
2,3,Weather
3,4,Third Party
4,5,Animal
5,6,Operation
6,7,OH Equipment Failure
7,8,Other
